In [1]:
import guidance

In [2]:
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

# Basic templating

Single variable


In [3]:
prompt = guidance('''What is {{example}}?''')
prompt

What is {{example}}?

In [4]:
prompt(example='truth')



What is truth?

Objects

In [5]:
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [{'name': 'truth', 'description': 'the state of being the case'},
         {'name': 'love', 'description': 'a strong feeling of affection'},]
prompt = guidance('''List of people:
{{#each people}}{{this}}
{{/each}}.
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')
prompt(people=people, ideas=ideas)

List of people:
John
Mary
Bob
Alice
.
List of ideas:
truth: the state of being the case
love: a strong feeling of affection

Generating

In [6]:
prompt = guidance('''The best thing about the beach is{{generate 'best' temperature=0.7}}''')
a = prompt()
a

The best thing about the beach is the fresh air and the sound of the waves. The ocean has a calming effect and the beautiful scenery is a perfect backdrop for relaxing and enjoying the day. The sun and sand provide a perfect place to soak up some Vitamin D and have fun. Additionally, the beach is a great spot for activities such as swimming, surfing, and beach volleyball.

Generating with `n>1`

In [7]:
prompt = guidance('''The best thing about the beach is{{generate 'best' n=3 temperature=0.7}}''')
a = prompt()
a

Error in prompt:  name 'echo' is not defined


NameError: name 'echo' is not defined

In [8]:
def aggregate(best):
    return '\n'.join(best)
prompt = guidance('''The best thing about the beach is{{#block hidden=True}}{{generate 'best' temperature=0.7 n=3}}{{/block}}
{{aggregate best}}''')
a = prompt(aggregate=aggregate)
a

Error in prompt:  name 'echo' is not defined


NameError: name 'echo' is not defined